<a href="https://colab.research.google.com/github/fsminako/text_rag/blob/main/5588654_rag_m2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RETRIEVAL AUGMENTED GENERATION (RAG) FOR MEDICAL RESEARCH

## Dataset Loading

The dataset used in this study will be medical research abstract sourced from the Arxiv library.

In [ ]:
#Installing necessary packages
!pip install arxiv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 899.5 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6049 sha256=022f41738d73e2522a7c86a6e8c88b2c1811031dcf01d5c7a97e346df70f843f
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k


In [ ]:
#Import packages
import arxiv
import numpy as np
import pandas as pd

In [ ]:
#Total observation that will be used in this study is 100 abstracts
n_records = 100

client = arxiv.Client()

search = arxiv.Search(
  query = "medical", #specifying the topic of the research
  max_results = n_records,
  sort_by = arxiv.SortCriterion.SubmittedDate #sorting the search based on the latest journal
)

results = client.results(search)

In [ ]:
#Abstract extraction process
abstracts = []

for r in client.results(search):
  abstracts.append(r.summary)

# Naming the column for the dataframe
df_data = {'abstract': abstracts}


In [ ]:
#Saving the extracted data as a data frame
df = pd.DataFrame(df_data)
df.head()

,abstract
0,The mining of adverse drug events (ADEs) is pi...
1,To address existing challenges with intravascu...
2,"In the past years, the amount of research on a..."
3,Many observational studies feature irregular l...
4,"In medical image analysis, the expertise scarc..."


## Data Cleaning

In [ ]:
import re

In [ ]:
def cleaning(text):
    if isinstance(text, str):
        url_pattern = re.compile(r'https://\S+|www\.\S+')
        text = url_pattern.sub('', text)
        text = re.sub(r"[’]", "'", text)
        text = re.sub(r"[^a-zA-Z\s'-]", "", text)
        text = ' '.join(text.split())
        text = text.lower()
    return text

df['abstract'] = df['abstract'].apply(lambda x: cleaning(x))

This process will:
*   remove url from the texts
*   converting " ’ " to " ' "
*   remove non-alphabetic character except ' and -
*   remove any extra whitespace (ensure that only single whitespace between each word)
*   convert all character into lowercase

In [ ]:
df.head()

,abstract
0,the mining of adverse drug events ades is pivo...
1,to address existing challenges with intravascu...
2,in the past years the amount of research on ac...
3,many observational studies feature irregular l...
4,in medical image analysis the expertise scarci...


In [ ]:
#Save the dataframe as a csv file
df["abstract"].to_csv("abstract.csv")

## Chunking

In [ ]:
#Installing required library
!pip install llama_index.core
!pip install llama_index.readers.file

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 3.4 MB/s eta 0:00:00


In [ ]:
#Importing the library
from llama_index.readers.file import FlatReader
from llama_index.core.node_parser import SentenceSplitter
from pathlib import Path

To separate the text into chunks, we will be using the SentenceSplitter from llama_index. This function will split the text in such a way that one sentence will not be separated into different chunk

In [ ]:
documents = FlatReader().load_data(Path("/content/abstract.csv"))

# we will limit to chunk size 100
parser = SentenceSplitter(chunk_size=100, chunk_overlap=10)
doc_nodes = parser.get_nodes_from_documents(documents)

In [ ]:
#Make a separate directory for the chunk data to ensure that it does not get mixed up with other data file
!mkdir -p '/content/chunk_data/'

In [ ]:
# Directory to save the individual chunk files
output_dir = Path("/content/chunk_data/")

# Save each chunk into a separate file
for i, node in enumerate(doc_nodes):
    output_file_path = output_dir / f"chunk_{i+1}.txt"
    with output_file_path.open("w", encoding="utf-8") as f:
        f.write(node.text)

print(f"Saved {len(doc_nodes)} chunks to {output_dir}")

Saved 135 chunks to /content/chunk_data


Each chunk will be saved into different data file. This process will later be helpful to identify which document is used to generate response during query processing

## Embedding

In [ ]:
#Installing necessary packages
!pip install langchain
!pip install langchain-community
!pip install sentence-transformers
!pip install llama-index-embeddings-langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Us

For our medical abstracts dataset, we will use PubMedBERT as our embedding model. PubMedBERT is trained on abstracts extracted from PubMed making it highly suitable for our dataset

In [ ]:
#Importing necessary library for the embeddings model
from langchain.embeddings import HuggingFaceEmbeddings

#Importing PubMedBERT from the hugging face library
embedding_model = HuggingFaceEmbeddings(model_name="microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/225k [00:00<?, ?B/s]

In [ ]:
#Configurating the default embedding model into our chosen embedding model
from llama_index.core import Settings
Settings.embed_model = embedding_model

## Indexing

In [ ]:
from llama_index.core import SimpleDirectoryReader

# Load all the documents in the chunk_data directory
reader = SimpleDirectoryReader("/content/chunk_data") # load documents from the chunk_data folder
documents = reader.load_data()
print(f"{len(documents)} documents are loaded")

135 documents are loaded


In [ ]:
#Installing necessary library
!pip install llama-index-vector-stores-chroma
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 586.7 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 5

In [ ]:
%%time
#Importing required packafes
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex

# Creating a medical_articles database
db = chromadb.PersistentClient(path="./medical_articles_db")

# Create a table inside the database called "medical-abstract"
chroma_collection = db.create_collection("medical-abstract")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Indexing the documents into the databse
vector_index = VectorStoreIndex.from_documents(
    documents,
    storage_context = storage_context,
    embed_model = embedding_model
)

# Printing the metadata
print(chroma_collection)

name='medical-abstract' id=UUID('a1808cb2-5f7a-45a0-9a45-16bcddfd956c') metadata=None tenant='default_tenant' database='default_database'
CPU times: user 1min 2s, sys: 329 ms, total: 1min 2s
Wall time: 1min 11s


## Prompt Template

Prompt template is crucial to engineer better response. We will use a customised prompt template from the llama library.
The prompt template that we use ensure that the LLM generate response as a medical expert but avoiding the use of medical terminology that is not generally used.

In [ ]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

#Prompt string for the LLM
qa_prompt_str = (
    "You are a medical expert, give responses to the following "
    "question: {query_str}. Do not use technical words, give easy "
    "to understand responses."
)

# Text QA Prompt
chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            "Always answer the question, even if the context isn't helpful."
        ),
    ),
    ChatMessage(role=MessageRole.USER, content=qa_prompt_str),
]

text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

## Query Processing and Response Generation

We will inegrate our RAG system with Ollama as the LLM. Ollama is designed to operate on large scale, making it effective on processing extensive data input.

In [ ]:
#Installing necessary packages
!pip install transformers
!pip install llama-index-llms-langchain
!pip install llama-index-llms-ollama
!pip install llama-index ipywidgets
!pip install llama_index.readers.web

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 24.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.1
    Uninstalling langchain-core-0.2.1:
      Successfully uninstalled langchain-core-0.2.1
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.2.0
    Uninstalling langchain-text-splitters-0.2.0:
      Successfully uninstalled langchain-text-splitters-0.2.0
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.2.1
    Uninstalling langchain-community-0.2.1:
      Successfully uninstalled langchain-community-0.2.1
  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.1
    Uninstalling langchain-0.2.1:
      Successfully uninstalled 

In [ ]:
!curl https://ollama.ai/install.sh | sed 's#https://ollama.ai/download#https://github.com/jmorganca/ollama/releases/download/v0.1.30#' | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 10406    0 10406    0     0  22032      0 --:--:-- --:--:-- --:--:-- 22000
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
#Setting up the model as a global variable
import os
OLLAMA_MODEL='phi:latest'

os.environ['OLLAMA_MODEL'] = OLLAMA_MODEL
!echo $OLLAMA_MODEL

#Importing LLM from Hugging face
from llama_index.llms.ollama import Ollama
llm = Ollama(model=OLLAMA_MODEL, request_timeout=12000.0) #the maximum processing time to generate response to a query

phi:latest


In [ ]:
import subprocess
import time

#Setting Ollama on the command
command = "nohup ollama serve&"

process = subprocess.Popen(command,
                            shell=True,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE)

time.sleep(5)

In [ ]:
#Testing the LLM without integrating with our vector database
!ollama run $OLLAMA_MODEL "Tell me a joke"

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏ 4.9 MB/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏ 4.9 MB/1.6 GB                  pulli

In [ ]:
#Settings Ollama as the default LLM
Settings.llm = llm

In [ ]:
#Input query for our RAG system
query = "What is the application of large language models in healthcare?"

In [ ]:
%%time
#Response processing
query_engine = vector_index.as_query_engine(
   text_qa_template=text_qa_template,
   llm=llm
)

response = query_engine.query(query)
response.response

CPU times: user 3.51 s, sys: 360 ms, total: 3.87 s
Wall time: 7min 40s


" Large Language Models (LMs) have been found to be very useful tools in the field of healthcare. They can help with tasks such as medical diagnosis, drug discovery, and even improving patient outcomes. For example, LMs can be used to analyze large amounts of medical data, such as electronic health records or social media posts, to identify patterns and predict potential diagnoses. They can also assist in the development of new drugs by predicting which compounds are most likely to be effective based on their molecular structures and properties. Additionally, LMs can help healthcare providers better understand patient needs by analyzing large amounts of unstructured data, such as free-text medical records or social media comments. Overall, LMs have the potential to revolutionize the way we approach healthcare, making it more accurate, efficient, and personalized for patients.\n\n\nImagine a scenario where you are a health data scientist working with a team of developers. You have been 

In [ ]:
#Retrieving the related documents to generate the response
response.metadata

{'2e629d53-1897-41c3-874a-e06176fb2228': {'file_path': '/content/chunk_data/chunk_7.txt',
  'file_name': 'chunk_7.txt',
  'file_type': 'text/plain',
  'file_size': 507,
  'creation_date': '2024-05-27',
  'last_modified_date': '2024-05-27'},
 'e3593a4f-eee7-497f-9832-682ecc4f2008': {'file_path': '/content/chunk_data/chunk_1.txt',
  'file_name': 'chunk_1.txt',
  'file_type': 'text/plain',
  'file_size': 601,
  'creation_date': '2024-05-27',
  'last_modified_date': '2024-05-27'}}

In [ ]:
#Retrieving the documents used to generate response
doc_nodes[95].text

'state-of-the-art baselines the code is available at\n35,with the widespread application of deep learning technology in medical image analysis how to effectively explain model decisions and improve diagnosis accuracy has become an urgent problem that needs to be solved attribution methods have become a key tool to help doctors better understand the diagnostic basis of models and they are used to explain and localize diseases in medical images however previous methods suffer from inaccurate and incomplete localization problems for fundus diseases with complex'

In [ ]:
doc_nodes[134].text

'by integrating these two components our proposed architecture improves accuracy in brain tumor segmentation we test our proposed model on the brats benchmark dataset and compare its performance with the state-of-the-art well-known segnet fcn-s and dense u-net architectures the results show that our proposed model outperforms the others in terms of the evaluated performance metrics'